# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [1]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install git+https://github.com/pytorch/tnt.git@master

  Cloning https://github.com/pytorch/tnt.git (to revision master) to c:\users\saket\appdata\local\temp\pip-req-build-pzrwuyug
  Stored in directory: C:\Users\saket\AppData\Local\Temp\pip-ephem-wheel-cache-gqp6fiua\wheels\17\05\ec\d05d051a225871af52bf504f5e8daf57704811b3c1850d0012
  Stored in directory: C:\Users\saket\AppData\Local\pip\Cache\wheels\70\19\a7\6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Stored in directory: C:\Users\saket\AppData\Local\pip\Cache\wheels\b1\c3\d6\9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchnet visdom torchfile
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/pytorch/tnt.git 'C:\Users\saket\AppData\Local\Temp\pip-req-build-pzrwuyug'


In [8]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt
import torch.optim as optim
use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: False


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25). 

For your convenience, we provide the dataloader for you.


In [18]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('C:/Users/saket/Downloads/hw6(2)/Question3/data', subset='train')
valSet = Dataset('C:/Users/saket/Downloads/hw6(2)/Question3/data', subset='val')
tstSet = Dataset('C:/Users/saket/Downloads/hw6(2)/Question3/data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=0, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=0, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=0, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [22]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75,100)
        self.recurrent_layer = nn.LSTM(100,100,1)
        self.classify_layer = nn.Linear(100,10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [13]:
################ 2nd To Do  (5 points)##################
dtype = torch.FloatTensor
optimizer = optim.SGD(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss().type(dtype)

In [25]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data)
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 10
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.3133,  Accuracy: 10.05
val Epoch: 0  , Loss: 2.3136,  Accuracy: 9.50
train Epoch: 1  , Loss: 2.3123,  Accuracy: 9.90
train Epoch: 2  , Loss: 2.3125,  Accuracy: 10.00
train Epoch: 3  , Loss: 2.3128,  Accuracy: 9.90
train Epoch: 4  , Loss: 2.3128,  Accuracy: 10.00
train Epoch: 5  , Loss: 2.3125,  Accuracy: 10.00
val Epoch: 5  , Loss: 2.3136,  Accuracy: 9.50
train Epoch: 6  , Loss: 2.3127,  Accuracy: 9.85
train Epoch: 7  , Loss: 2.3123,  Accuracy: 10.00
train Epoch: 8  , Loss: 2.3126,  Accuracy: 10.10
train Epoch: 9  , Loss: 2.3132,  Accuracy: 10.05


## Submit your results 

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set. 


### Testing the model and reporting the results
Test the model on the testing set and save the results as a .csv file. 
submit the results.csv file generated by predict_on_test(). Also mention the best performance on the Validation set, and submit the corresponding results csv file which results in the best performance. 
################ 3rd To Do  (15 points) ###############


In [26]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)

In [66]:
# sequence classification model
class SequenceClassify1(nn.Module):
    def __init__(self):
        super(SequenceClassify1, self).__init__()
        
        self.project_layer = nn.Sequential(
            nn.Conv2d(1, 128,kernel_size = 3, stride = 1),
            nn.ReLU(inplace = True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size =2, stride = 2, padding =0),
            nn.Conv2d(128,256,kernel_size = 2, stride = 1),
            nn.ReLU(inplace = True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size = 2,stride = 2, padding = 0),
            Flatten(),
            nn.ReLU(inplace = True),
            nn.Linear(8704,2048),
            nn.Dropout(p = 0.2),
            nn.Linear(2048,100),
        )
        self.recurrent_layer = nn.LSTM(100,100,1)
        self.classify_layer = nn.Linear(100,10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs = self.project_layer(input.view(50, 1, 15, 75))
        rnn_outputs, (hn, cn) = self.recurrent_layer(rnn_outputs.view(50,1,100))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model1 = SequenceClassify1()

In [67]:
dtype = torch.FloatTensor
optimizer1 = optim.Adam(model1.parameters(), lr = 1e-4)
criterion1 = nn.CrossEntropyLoss()

In [68]:
num_epochs = 10
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model1, criterion1, e, True, optimizer1)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model1, criterion1, e, False, None)   
    

train Epoch: 0  , Loss: 2.2141,  Accuracy: 18.25
val Epoch: 0  , Loss: 2.1615,  Accuracy: 21.00
train Epoch: 1  , Loss: 1.9210,  Accuracy: 42.25
train Epoch: 2  , Loss: 1.6402,  Accuracy: 57.60
train Epoch: 3  , Loss: 1.4284,  Accuracy: 67.20
train Epoch: 4  , Loss: 1.2820,  Accuracy: 69.70
train Epoch: 5  , Loss: 1.1678,  Accuracy: 73.50
val Epoch: 5  , Loss: 1.1510,  Accuracy: 74.50
train Epoch: 6  , Loss: 1.0775,  Accuracy: 75.55
train Epoch: 7  , Loss: 0.9998,  Accuracy: 77.55
train Epoch: 8  , Loss: 0.9185,  Accuracy: 79.45
train Epoch: 9  , Loss: 0.8530,  Accuracy: 81.85


In [70]:
# Use your best model to generate results on test set and validation set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model1.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results_q3.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i].item())+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model1, tstLD)
print(count)

500


In [71]:
# Use your best model to generate results on test set and validation set.

# generate csv file for val set
def predict_on_test(model, data_loader):
    model1.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results_q3_val.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i].item())+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model1, valLD)
print(count)

400


In [72]:
run_epoch(valLD, model1, criterion1, 1, False, None)

val Epoch: 1  , Loss: 0.9728,  Accuracy: 75.25


75.25

## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epoches) and any visualizations or graphs that you make in the process of training and evaluating your network.



1) For my network I decided to use the LSTM architecture. 

2) Before feeding into the LSTM, I used two convolutional layers with batch normalization and max pooling for my input.

3) I also used a couple of linear layers to map to the final input before feeding it into the LSTM network.

4) Using dropout layers between the linear layers helped reduce overfitting and my validation accuracy was also improving steadily with each epoch.

5) I used an Adam Optimizer and cross entropy loss as this combination proved to be the most fruitful. I also tried various loss functions such as NLL Losss and vaious optimizers such as SGD. 

6) Training my network for 10 epochs significantly increased the training erorr, although, due to time consumption, I decided not to train for many more epochs. However, doing so may have led to further convergence of the training loss.